In [1]:
import pandas as pd

def get_executions_df(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    return df.groupby("date").sum()[["realized_pnl", "px_side"]]

get_executions_df("../../execution-477837024.csv")

,realized_pnl,px_side
date,,
2022-03-24,717.28,145.17


In [2]:
def get_executions_ungroup(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    cols = ["quantity", "realized_pnl", "px_side"]

    return df, df.groupby("date").sum()[cols], df.groupby("date").count()[cols]

df_all, df_sum, df_count = get_executions_ungroup("../../execution-477837024.csv")
df_all

,Unnamed: 0,contract,time_completed,side,quantity,avg_price,realized_pnl,epoch_sec,profit,loss,...,avg_pnl_loss,avg_pnl_rr,avg_pnl_ewr,total_px_profit,avg_px_profit,total_px_loss,avg_px_loss,avg_px_rr,avg_px_ewr,date
1,1,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-24 03:32:42,BOT,1,14567.00,-85.1400,1.648111e+09,0,1,...,-85.14,NaN,NaN,0.00000,NaN,-42.5700,-42.570000,NaN,NaN,2022-03-24
6,6,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-24 05:00:13,SLD,4,14539.25,-133.0600,1.648116e+09,0,2,...,-109.10,NaN,NaN,0.00000,NaN,-59.2025,-29.601250,NaN,NaN,2022-03-24
9,9,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-24 06:09:13,SLD,1,14564.00,103.1100,1.648120e+09,1,2,...,-109.10,0.945096,0.514113,51.55500,51.555000,0.0000,-29.601250,1.741649,0.364744,2022-03-24
13,13,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-24 10:01:13,SLD,1,14565.00,126.7975,1.648134e+09,2,2,...,-109.10,1.053655,0.486937,114.95375,57.476875,0.0000,-29.601250,1.941704,0.339939,2022-03-24
14,14,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-24 10:13:02,SLD,3,14567.00,392.3925,1.648135e+09,3,2,...,-109.10,1.901314,0.344671,180.35250,60.117500,0.0000,-29.601250,2.030911,0.329934,2022-03-24
17,17,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-24 11:29:18,SLD,2,14594.00,-115.2800,1.648139e+09,3,3,...,-111.16,1.866079,0.348909,0.00000,60.117500,-88.0225,-29.340833,2.048936,0.327983,2022-03-24
19,19,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-24 11:34:05,BOT,2,14592.00,12.7200,1.648140e+09,4,3,...,-111.16,1.428167,0.411833,183.53250,45.883125,0.0000,-29.340833,1.563798,0.390046,2022-03-24
25,25,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-24 14:42:23,SLD,5,14697.25,92.3000,1.648151e+09,5,3,...,-111.16,1.308600,0.433163,192.76250,38.552500,0.0000,-29.340833,1.313954,0.432161,2022-03-24
29,29,"477837024,MNQ,FUT,20220617,0,,2,GLOBEX,,USD,MN...",2022-03-24 15:02:04,SLD,4,14763.00,323.4400,1.648152e+09,6,3,...,-111.16,1.575447,0.388282,233.19250,38.865417,0.0000,-29.340833,1.324619,0.430178,2022-03-24


In [3]:
df_sum


,quantity,realized_pnl,px_side
date,,,
2022-03-24,23,717.28,145.17


In [4]:
df_count


,quantity,realized_pnl,px_side
date,,,
2022-03-24,9,9,9


In [5]:
def get_executions_ungroup(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    return df[df["realized_pnl"] > 0].groupby("date").sum()[["realized_pnl"]], df[df["realized_pnl"] < 0].groupby("date").sum()[["realized_pnl"]]

df_pos, df_neg = get_executions_ungroup("../../execution-477837024.csv")

In [6]:
df_pos


,realized_pnl
date,
2022-03-24,1050.76


In [7]:
df_neg

,realized_pnl
date,
2022-03-24,-333.48
